In [1]:
from sidecar import Sidecar
import keplergl
import pandas as pd
import geopandas as gpd

In [2]:
def convert_to_ijv(mtx, mode):
    ijv = (
        mtx
        .reset_index(names=['origin'])
        .melt(id_vars=['origin'], 
              value_vars=mtx.index, 
              var_name='destination', 
              value_name=f'{mode}_{purpose}')
        .set_index(['origin', 'destination'])
    )
    
    return ijv

In [3]:
def merge_od_centroid(mt_ijv, zone_shp):
    mt_ijv = (
        mt_ijv
        .fillna(0)
        .loc[mt_ijv.sum(axis=1)>0]
    )
    
    mt_ijv = (
        mt_ijv
        .reset_index()
        .merge(zone_shp.drop('geometry', axis=1), how='left', left_on='origin', right_on='ART3_2B_Ne')
        .rename(columns={col: f'from_{col}' for col in zone_shp.columns})
        .merge(zone_shp.drop('geometry', axis=1), how='left', left_on='destination', right_on='ART3_2B_Ne')
        .rename(columns={col: f'to_{col}' for col in zone_shp.columns})
        .dropna(subset=['from_ART3_2B_Ne', 'to_ART3_2B_Ne'])
    ) 
    
    return mt_ijv

In [70]:
config = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'irodas',
     'type': 'arc',
     'config': {'dataId': 'demand_matrix',
      'label': 'from centroid -> to centroid arc',
      'color': [187, 0, 0],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'from_centroid_lat',
       'lng0': 'from_centroid_lon',
       'lat1': 'to_centroid_lat',
       'lng1': 'to_centroid_lon'},
      'isVisible': True,
      'visConfig': {'opacity': 0.2,
       'thickness': 2,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 20],
       'targetColor': [255, 251, 152]},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'PTPerson_AllPurp',
       'type': 'integer'},
      'colorScale': 'quantile',
      'sizeField': {'name': 'PTPerson_AllPurp', 'type': 'integer'},
      'sizeScale': 'linear'}},
    {'id': 't6w4ez8',
     'type': 'geojson',
     'config': {'dataId': 'zones',
      'label': 'new layer',
      'color': [18, 92, 119],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': False,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'demand_matrix': [{'name': 'origin',
        'format': None},
       {'name': 'destination', 'format': None},
       {'name': 'CarPerson_HBW', 'format': None},
       {'name': 'PTPerson_HBW', 'format': None},
       {'name': 'CarPerson_HBE', 'format': None}],
      'zones': [{'name': 'ART3_2B_Ne', 'format': None},
       {'name': 'Zone', 'format': None},
       {'name': 'Area', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': False},
    'brush': {'size': 0.5, 'enabled': True},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': -18.433789621530053,
   'dragRotate': True,
   'latitude': -36.86262844956161,
   'longitude': 174.74352568506777,
   'pitch': 5.777654402162497,
   'zoom': 10.120369203891174,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

# Convert Matrix to IJV Format

In [80]:
zone_shp = gpd.read_file("input/AWHC zoning system v1.geojson").to_crs('EPSG:4326')

zone_shp['centroid_lat'] = zone_shp['geometry'].centroid.y
zone_shp['centroid_lon'] = zone_shp['geometry'].centroid.x
is_airport = zone_shp['ART3_2B_Ne']==478

international = zone_shp.loc[is_airport].reset_index(drop=True)
international['ART3_2B_Ne'] = 597

domestic = zone_shp.loc[is_airport].reset_index(drop=True)
domestic['ART3_2B_Ne'] = 598

zone_shp = pd.concat([zone_shp, international, domestic], ignore_index=True)



/tmp/ipykernel_6498/3793826346.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zone_shp['centroid_lat'] = zone_shp['geometry'].centroid.y
/tmp/ipykernel_6498/3793826346.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zone_shp['centroid_lon'] = zone_shp['geometry'].centroid.x


In [49]:
%%time

scen_id = '20_Reference_pwclu'
purposes = ['HBW', 'HBE', 'HBO', 'AllPurp']

mt_ijv = pd.DataFrame()
for purpose in purposes:
    file_name = f"sectors 2065/{scen_id}/0205_LRT_CC2M_MoT_stage5_E46b_2105_MSM demand sectoring_{purpose}_65120_05thOct22.xlsx"
    
    car_person_mtx = pd.read_excel(file_name, sheet_name='Car Person MSM', index_col=0)
    car_person_ijv = convert_to_ijv(car_person_mtx, 'CarPerson')
    
    pt_person_mtx = pd.read_excel(file_name, sheet_name='PT Person MSM', index_col=0)
    pt_person_ijv = convert_to_ijv(pt_person_mtx, 'PTPerson')
    
    mt_ijv = pd.concat([mt_ijv, car_person_ijv, pt_person_ijv], axis=1)
    
ref_mt_ijv = (
    merge_od_centroid(mt_ijv, zone_shp)
    .dropna(subset=['from_Zone', 'to_Zone'])
)

ref_mt_ijv.to_csv(f'sectors 2065/outputs-ijv/{scen_id}_msm_matrix_ijv.csv')

CPU times: user 24.9 s, sys: 159 ms, total: 25 s
Wall time: 25 s


In [50]:
%%time

scen_id = '14_DoMin'
purposes = ['HBW', 'HBE', 'HBO', 'AllPurp']

mt_ijv = pd.DataFrame()
for purpose in purposes:
    file_name = f"sectors 2065/{scen_id}/0205_LRT_CC2M_MoT_stage5_E46b_2105_MSM demand sectoring_{purpose}_65114_05thOct22.xlsx"
    
    car_person_mtx = pd.read_excel(file_name, sheet_name='Car Person MSM', index_col=0)
    car_person_ijv = convert_to_ijv(car_person_mtx, 'CarPerson')
    
    pt_person_mtx = pd.read_excel(file_name, sheet_name='PT Person MSM', index_col=0)
    pt_person_ijv = convert_to_ijv(pt_person_mtx, 'PTPerson')
    
    mt_ijv = pd.concat([mt_ijv, car_person_ijv, pt_person_ijv], axis=1)

dm_mt_ijv = (
    merge_od_centroid(mt_ijv, zone_shp)
    .dropna(subset=['from_Zone', 'to_Zone'])
)

dm_mt_ijv.to_csv(f'sectors 2065/outputs-ijv/{scen_id}_msm_matrix_ijv.csv')

CPU times: user 25 s, sys: 109 ms, total: 25.1 s
Wall time: 25.1 s


In [81]:
wl = keplergl.KeplerGl(height=900, config=config)
wl.add_data(data=zone_shp, name='zones')
wl.add_data(data=ref_mt_ijv, name='demand_matrix')
wl.save_to_html(file_name='sectors 2065/outputs-keplergl/20_Reference_pwclu - demand matrix.html')

wl.add_data(data=dm_mt_ijv, name='demand_matrix')
wl.save_to_html(file_name='sectors 2065/outputs-keplergl/14_DoMin - demand matrix.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to sectors 2065/outputs-keplergl/20_Reference_pwclu - demand matrix.html!
Map saved to sectors 2065/outputs-keplergl/14_DoMin - demand matrix.html!
